<a href="https://colab.research.google.com/github/MpRonald/Deep-Learning/blob/main/PyTorch_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [46]:
# !pip install torch==1.4.0
!pip install skorch -q

     |████████████████████████████████| 155 kB 33.0 MB/s 


In [47]:
import pandas as pd
import torch
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from skorch import NeuralNetRegressor
from torch import nn, optim
import torch.nn.functional as F
torch.__version__

'1.12.0+cu113'

# Dataset and Pre Processing

In [4]:
np.random.seed(42)
torch.manual_seed(42)

In [5]:
data = pd.read_csv('/content/drive/MyDrive/Datasets/autos.csv', encoding='ISO-8859-1')
data.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [6]:
data.shape

(371528, 20)

In [7]:
data.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [8]:
dataset = data[['price', 'abtest', 'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'kilometer', 'monthOfRegistration', 'fuelType', 'brand', 'notRepairedDamage']]

In [9]:
dataset.shape

(371528, 12)

In [10]:
low_prices = dataset.loc[dataset.price < 200]
low_prices.shape[0]

17470

In [11]:
# drop the low prices
dataset = dataset[dataset.price > 200]
dataset.shape

(352072, 12)

In [12]:
high_price = dataset[dataset.price > 50000]
high_price.shape[0]

1562

In [13]:
dataset = dataset[dataset.price < 50000]
dataset.shape

(350490, 12)

In [14]:
dataset.isnull().sum()

price                      0
abtest                     0
vehicleType            31179
yearOfRegistration         0
gearbox                15522
powerPS                    0
model                  16899
kilometer                  0
monthOfRegistration        0
fuelType               27116
brand                      0
notRepairedDamage      62389
dtype: int64

In [15]:
dataset.vehicleType.value_counts()

limousine     92540
kleinwagen    74990
kombi         65237
bus           29557
cabrio        22015
coupe         17671
suv           14262
andere         3039
Name: vehicleType, dtype: int64

In [16]:
dataset.gearbox.value_counts()

manuell      260645
automatik     74323
Name: gearbox, dtype: int64

In [17]:
dataset.model.value_counts()

golf               28438
andere             24771
3er                19764
polo               11965
corsa              11327
                   ...  
elefantino             6
rangerover             6
serie_3                3
serie_1                1
discovery_sport        1
Name: model, Length: 251, dtype: int64

In [18]:
dataset.fuelType.value_counts()

benzin     211628
diesel     105493
lpg          5200
cng           553
hybrid        266
andere        145
elektro        89
Name: fuelType, dtype: int64

In [19]:
dataset.notRepairedDamage.value_counts()

nein    256625
ja       31476
Name: notRepairedDamage, dtype: int64

In [20]:
values = {'vehicleType':'limousine', 'gearbox':'manuell', 'model':'golf',
          'fuelType':'benzin', 'notRepairedDamage':'nein'}

In [21]:
dataset = dataset.fillna(value=values)

In [22]:
dataset.isnull().sum()

price                  0
abtest                 0
vehicleType            0
yearOfRegistration     0
gearbox                0
powerPS                0
model                  0
kilometer              0
monthOfRegistration    0
fuelType               0
brand                  0
notRepairedDamage      0
dtype: int64

In [23]:
dataset.duplicated().sum()

20196

In [24]:
dataset.drop_duplicates(inplace=True)
dataset.duplicated().sum()

0

In [25]:
predict = dataset.iloc[:, 1:13].values
real_price = dataset.iloc[:,0].values

In [26]:
predict.shape, real_price.shape

((330294, 11), (330294,))

In [27]:
predict = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [0,1,3,5,8,9,10])], 
                           remainder = 'passthrough').fit_transform(predict).toarray()

In [28]:
predict.shape[1]

316

In [29]:
predict = torch.tensor(predict, dtype=torch.float)
real_price = torch.tensor(real_price, dtype=torch.float).view(-1,1)

# Model

In [30]:
n_input = predict.shape[1]
n_hidden = n_input//2
n_output = real_price.shape[1]
n_input, n_hidden, n_output

(316, 158, 1)

In [31]:
regression = nn.Sequential(
    nn.Linear(n_input, n_hidden),
    nn.ReLU(),
    nn.Linear(n_hidden, n_hidden),
    nn.ReLU(),
    nn.Linear(n_hidden, n_output),
)

In [32]:
criterion = nn.L1Loss()

In [33]:
optmizer = optim.Adam(regression.parameters())

In [34]:
data_torch = torch.utils.data.TensorDataset(predict, real_price)
train_loader = torch.utils.data.DataLoader(data_torch, batch_size=300, shuffle=True)

# Training

In [35]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [36]:
regression.to(device)

Sequential(
  (0): Linear(in_features=316, out_features=158, bias=True)
  (1): ReLU()
  (2): Linear(in_features=158, out_features=158, bias=True)
  (3): ReLU()
  (4): Linear(in_features=158, out_features=1, bias=True)
)

In [37]:
for epoch in range(100):
    run_loss = 0.
    run_mae = 0.

    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optmizer.zero_grad()

        outputs = regression.forward(inputs)
        mae = F.l1_loss(outputs, labels).item()

        run_mae += mae

        loss = criterion(outputs, labels)
        loss.backward()
        optmizer.step()

        run_loss += loss.item()

        print(f"Epoch: {epoch+1} --- Loop {i+1}/{len(real_price)//300} --- Loss: {loss} --- MAE: {mae}")
    
    print(f"Epoch: {epoch+1} finalized --- Loss: {run_loss/len(train_loader)} --- MAE: {run_mae/len(train_loader)}")

A saída de streaming foi truncada nas últimas 5000 linhas.
Epoch: 96 --- Loop 511/1100 --- Loss: 1952.1285400390625 --- MAE: 1952.1285400390625
Epoch: 96 --- Loop 512/1100 --- Loss: 2013.2808837890625 --- MAE: 2013.2808837890625
Epoch: 96 --- Loop 513/1100 --- Loss: 2205.278076171875 --- MAE: 2205.278076171875
Epoch: 96 --- Loop 514/1100 --- Loss: 1909.0208740234375 --- MAE: 1909.0208740234375
Epoch: 96 --- Loop 515/1100 --- Loss: 1921.5919189453125 --- MAE: 1921.5919189453125
Epoch: 96 --- Loop 516/1100 --- Loss: 1886.152099609375 --- MAE: 1886.152099609375
Epoch: 96 --- Loop 517/1100 --- Loss: 2173.779541015625 --- MAE: 2173.779541015625
Epoch: 96 --- Loop 518/1100 --- Loss: 2099.278076171875 --- MAE: 2099.278076171875
Epoch: 96 --- Loop 519/1100 --- Loss: 2226.28955078125 --- MAE: 2226.28955078125
Epoch: 96 --- Loop 520/1100 --- Loss: 2204.685791015625 --- MAE: 2204.685791015625
Epoch: 96 --- Loop 521/1100 --- Loss: 2202.499267578125 --- MAE: 2202.499267578125
Epoch: 96 --- Loop 522

# Model Validate

In [38]:
predict = regression.forward(predict.to(device))

In [39]:
predict

tensor([[  805.3906],
        [ 7393.1069],
        [ 8990.4697],
        ...,
        [ 3981.6343],
        [ 2081.7910],
        [26096.4141]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [40]:
real_price

tensor([[  480.],
        [18300.],
        [ 9800.],
        ...,
        [ 9200.],
        [ 3400.],
        [28990.]])

In [41]:
predict.mean(), real_price.mean()

(tensor(4065.7429, device='cuda:0', grad_fn=<MeanBackward0>),
 tensor(5728.8257))